# Asking ChatGPT science questions via the API
Very basic notebook that uses the ChatGPT API to answer some questions from our science homework. To get this far took 2¢.

In [1]:
import pandas as pd
import os                                                                                                                                                                                                          
from dotenv import load_dotenv
from pathlib import Path
import openai

In [2]:
load_dotenv(Path(".env"))
openai.api_key = os.environ.get("CHAT_API")

qa = pd.read_csv("data/train.csv")
qa.head(3)

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A


In [18]:
qa["gpt_prompt"] = qa["prompt"] + " A)" + qa["A"]+ " B)" + qa["B"]+ " C)" + qa["C"] + " D)" + qa["D"] + " E" + qa["E"] # Reformat into single line
qa_train = qa["gpt_prompt"][0:5] # Quick cheat to peel some data off to use
qa_x = qa["answer"][0:5]

Loop around questions and ask each to chatGPT.

In [16]:
output = []

for i in range(len(qa_train)):
  response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
      {'role': 'system', 'content': 'You are answering multiple choice science questions.'},
      {'role': 'user', 'content': qa_train.tolist()[i]},
    ]
  )
  
  chat_response = response['choices'][0]['message']['content']
  output.append(chat_response)

print(output)

['D) MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2.', 'A) Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times exhibits similarity to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.', 'B) The triskeles symbol is a representation of three interlinked spirals, which was adopted as an emblem by the rulers of Syracuse. Its usage in modern flags of Sicily has its origins in the ancient Greek name for the island, Trinacria, which means "Sicily with three corners." The head of Medusa at the center is a representation of the island\'s rich cultural heritage.', 'A) Regularizing the mass-energy of an electron with a finite radius can theoretically simplify calculations involving infinities

Again, take a shortcut to check how performance looks. Inital signs aren't great.

In [17]:
letter_answers = [item[0] for item in output]
print(letter_answers, qa_x.tolist())

['D', 'A', 'B', 'A', 'B'] ['D', 'A', 'A', 'C', 'D']
